In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium import webdriver
from selenium.webdriver.support.ui import Select
import requests

from bs4 import BeautifulSoup
import re as re
import time
import pandas as pd
import numpy as np

import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nltk.corpus import stopwords

from wordcloud import WordCloud
from langdetect import detect
from textblob import TextBlob
import spacy

In [2]:
def get_bbc():
    driver = webdriver.Chrome()
    url= 'https://www.bbc.com/news'
    driver.get(url)
    time.sleep(5)
    cookies_accept = driver.find_element(By.CLASS_NAME, 'fc-button').click()
    search_bar     = driver.find_element(By.CLASS_NAME, 'ux-v5').click()
    search_bar     = driver.find_element(By.ID, 'search-input')
    search_bar.send_keys('Ukraine war')
    search_bar.send_keys(Keys.RETURN)
    
    lnks           = driver.find_elements(By.TAG_NAME,"a")
    lst_bbc        = [lnk.get_attribute('href') for lnk in lnks]
    lst_bbc_ukr    = [url for url in lst_bbc if '/news/' in url]
 
    return lst_bbc_ukr

In [3]:
def bbc_list(lst_bbc_ukr):
    dict_article_bbc={}
    for url in lst_bbc_ukr:
        html = requests.get(url)
        soup = BeautifulSoup(html.content, "html.parser")
        article=soup.getText().replace('\'', "´").strip()
        dict_article_bbc[url]=article
    return dict_article_bbc

In [4]:
def descriptives(dict_article_bbc):
    length=[len(articles) for articles in dict_article_bbc.values()]
    lst_polarity = [round(TextBlob(article).sentiment.polarity,5) for article in dict_article_bbc.values()]
    lst_subjectivity = [round(TextBlob(article).sentiment.subjectivity,5) for article in dict_article_bbc.values()]
    ukraine=[articles.lower().count('ukraine') for articles in dict_article_bbc.values()]
    war=[articles.lower().count('war') for articles in dict_article_bbc.values()]
    russia=[articles.lower().count('russia') for articles in dict_article_bbc.values()]
    putin = [articles.lower().count('putin') for articles in dict_article_bbc.values()]
    zelensky= [articles.lower().count('zelensky') for articles in dict_article_bbc.values()]
    dict_length= {'polarity':lst_polarity,'subjetivity':lst_subjectivity,'length':length,'ukraine':ukraine,'war':war, 'russia':russia,'putin':putin,'zelensky':zelensky}
    df_descriptive= pd.DataFrame(dict_length)
    return df_descriptive

In [5]:
def create_df_bbc(dict_article_bbc):
    dict_article_bbc = bbc_list(lst_bbc_ukr)
    idx= {'article'}
    df_bbc= pd.DataFrame(dict_article_bbc,idx).transpose().reset_index()
    df_bbc= df_bbc.rename(columns={"index": "link"})
    return df_bbc

In [6]:
lst_bbc_ukr = get_bbc()

In [7]:
len(lst_bbc_ukr)

9

In [8]:
dict_article_bbc=bbc_list(lst_bbc_ukr)
len(dict_article_bbc)

9

In [9]:
df_descriptive =descriptives(dict_article_bbc)


In [10]:
df_bbc = create_df_bbc(dict_article_bbc)

In [11]:
df_bbc_together = df_bbc.join(df_descriptive)

In [12]:
df_bbc_together

,link,article,polarity,subjetivity,length,ukraine,war,russia,putin,zelensky
0,https://www.bbc.co.uk/news/uk-scotland-glasgow...,"Ukraine war: I told friends I´d play football,...",0.06872,0.47202,8145,23,9,6,1,0
1,https://www.bbc.co.uk/news/uk-northern-ireland...,Ukraine War: ‘I saw bodies just lying in the s...,0.12368,0.25658,3625,14,10,2,0,4
2,https://www.bbc.co.uk/news/uk-england-london-6...,Ukraine war: Road outside Russian Embassy rena...,-0.02608,0.30426,5582,12,10,11,3,0
3,https://www.bbc.co.uk/news/uk-england-derbyshi...,Ukraine war: ´Settling somewhere new is hard -...,0.05441,0.45468,4059,13,9,4,0,4
4,https://www.bbc.co.uk/news/world-europe-64740093,Ukraine war a violation of international law -...,0.17721,0.31735,3225,13,11,3,0,4
5,https://www.bbc.co.uk/news/uk-england-northamp...,Ukraine war: ´We have belief in our president ...,0.02955,0.43767,8175,22,13,8,2,0
6,https://www.bbc.co.uk/news/world-europe-64762219,Ukraine war: Zelensky wants Xi Jinping meeting...,0.02912,0.37101,8285,24,18,21,8,10
7,https://www.bbc.co.uk/news/uk-england-wiltshir...,Ukraine war: Refugee friendships forged in Swi...,0.03151,0.43237,6252,7,5,2,1,0
8,https://www.bbc.co.uk/news/help-41670342,Learn how the BBC is working to strengthen tru...,0.14551,0.44835,9558,1,1,0,0,0
